In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import os
import random
import stellargraph as sg
from stellargraph.mapper import Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

data_dir = "data/"
walk_length = 3
number_of_walks = 3
batch_size = 32
epochs = 50
layer_sizes = [16]
kfold = 10

# Read edge list
edgelist = pd.read_csv(os.path.join(data_dir, "pair_Sa"),
                       sep="\t", header=None, names=["source", "target", "label", "eo1", "eo2"])

# Read node list
file_nodes = pd.read_csv(os.path.join(data_dir, "pair_content_Sa"), sep="\t")
nodes = file_nodes.set_index('ID')
feats = nodes.columns[:-1]  # Exclude label column

# Create graph
G_all_nx = nx.from_pandas_edgelist(edgelist, edge_attr="label")
nx.set_node_attributes(G_all_nx, "label", "label")
G_all = sg.StellarGraph.from_networkx(G_all_nx, node_features=nodes[feats])

# Store probabilities
synergy_probs = {"Hadamard": [], "L1-norm": [], "L2-norm": [], "Average": [], "Classification-based": []}
antagonistic_probs = {"Hadamard": [], "L1-norm": [], "L2-norm": [], "Average": [], "Classification-based": []}
lbl_p_synergistic, lbl_p_antagonistic = [], []

# Train Attri2Vec model
generator = Attri2VecNodeGenerator(G_all, batch_size)
attri2vec = Attri2Vec(layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None)
x_inp, x_out = attri2vec.in_out_tensors()
embedding_model = keras.Model(inputs=x_inp, outputs=x_out)

for k in range(kfold):
    print(f"Processing Fold {k+1}/{kfold}...")

    # Read train and test interaction data
    edge_t = pd.read_csv(os.path.join(data_dir, f"{k+1}_t"), sep="\t",
                         header=None, names=["source", "target", "label", "eo1", "eo2"])
    edge_p = pd.read_csv(os.path.join(data_dir, f"{k+1}_p"), sep="\t",
                         header=None, names=["source", "target", "label", "eo1", "eo2"])

    # Convert labels from [1,2,3] → [0,1,2]
    edge_t["label"] = edge_t["label"] - 1
    edge_p["label"] = edge_p["label"] - 1

    # Generate embeddings
    node_ids = nodes.index
    node_gen = generator.flow(node_ids)
    emb = embedding_model.predict(node_gen, workers=4, verbose=1)

    # Function to extract edge features
    def extract_features(edge_data):
        feat_h, feat_L1, feat_L2, feat_av, feat_cmp_av = [], [], [], [], []
        for i in range(len(edge_data)):
            try:
                n1 = np.where(nodes.index == edge_data["source"][i])[0][0]
                n2 = np.where(nodes.index == edge_data["target"][i])[0][0]

                feat_h.append(emb[n1] * emb[n2])  # Hadamard
                feat_L1.append(np.abs(emb[n1] - emb[n2]))  # L1-norm
                feat_L2.append((emb[n1] - emb[n2]) ** 2)  # L2-norm
                feat_av.append((emb[n1] + emb[n2]) / 2)  # Average
                feat_cmp_av.append((nodes.loc[edge_data["source"][i], feats] +
                                    nodes.loc[edge_data["target"][i], feats]) / 2)  # Classification-based
            except IndexError:
                print(f"Skipping invalid edge: {edge_data.iloc[i].to_dict()}")
        
        return np.array(feat_h), np.array(feat_L1), np.array(feat_L2), np.array(feat_av), np.array(feat_cmp_av)

    # Extract features
    h_feat_t, L1_feat_t, L2_feat_t, av_feat_t, cmp_av_feat_t = extract_features(edge_t)
    h_feat_p, L1_feat_p, L2_feat_p, av_feat_p, cmp_av_feat_p = extract_features(edge_p)

    # Prepare labels
    lbl_p = edge_p["label"].values
    lbl_p_synergistic.extend((lbl_p == 1).astype(int))
    lbl_p_antagonistic.extend((lbl_p == 2).astype(int))

    # Function to train CNN
    def train_cnn(train_X, train_y, test_X):
        # Convert data to float32
        train_X = np.array(train_X, dtype=np.float32).reshape(train_X.shape[0], train_X.shape[1], 1)
        test_X = np.array(test_X, dtype=np.float32).reshape(test_X.shape[0], test_X.shape[1], 1)
        train_y = np.array(train_y, dtype=np.float32)

        model = Sequential([
            Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(train_X.shape[1], 1)),
            Flatten(),
            Dense(8, activation='relu'),
            Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
        model.fit(train_X, train_y, epochs=10, batch_size=32, verbose=0)
        
        pred_proba = model.predict(test_X).flatten()
        return pred_proba

    # Train CNN for each feature method
    synergy_probs["Hadamard"].append(train_cnn(h_feat_t, (edge_t["label"] == 1).astype(int), h_feat_p))
    synergy_probs["L1-norm"].append(train_cnn(L1_feat_t, (edge_t["label"] == 1).astype(int), L1_feat_p))
    synergy_probs["L2-norm"].append(train_cnn(L2_feat_t, (edge_t["label"] == 1).astype(int), L2_feat_p))
    synergy_probs["Average"].append(train_cnn(av_feat_t, (edge_t["label"] == 1).astype(int), av_feat_p))
    synergy_probs["Classification-based"].append(train_cnn(cmp_av_feat_t, (edge_t["label"] == 1).astype(int), cmp_av_feat_p))

    antagonistic_probs["Hadamard"].append(train_cnn(h_feat_t, (edge_t["label"] == 2).astype(int), h_feat_p))
    antagonistic_probs["L1-norm"].append(train_cnn(L1_feat_t, (edge_t["label"] == 2).astype(int), L1_feat_p))
    antagonistic_probs["L2-norm"].append(train_cnn(L2_feat_t, (edge_t["label"] == 2).astype(int), L2_feat_p))
    antagonistic_probs["Average"].append(train_cnn(av_feat_t, (edge_t["label"] == 2).astype(int), av_feat_p))
    antagonistic_probs["Classification-based"].append(train_cnn(cmp_av_feat_t, (edge_t["label"] == 2).astype(int), cmp_av_feat_p))

# Compute AUC scores
print("\nAUC (Synergistic vs. Rest):")
for method in synergy_probs:
    print(f"{method}: {roc_auc_score(lbl_p_synergistic, np.hstack(synergy_probs[method])):.4f}")

print("\nAUC (Antagonistic vs. Rest):")
for method in antagonistic_probs:
    print(f"{method}: {roc_auc_score(lbl_p_antagonistic, np.hstack(antagonistic_probs[method])):.4f}")


Processing Fold 1/10...


4/4 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 31ms/step


...

1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 31ms/step

AUC (Synergistic vs. Rest):
Hadamard: 0.4275
L1-norm: 0.5799
L2-norm: 0.4926
Average: 0.4576
Classification-based: 0.4580

AUC (Antagonistic vs. Rest):
Hadamard: 0.4258
L1-norm: 0.4689
L2-norm: 0.4132
Average: 0.4801
Classification-based: 0.5411
